# Preparing to play

This notebook has my work towards an implementation of the play function in play.py

Initial imports

In [44]:
import pandas as pd
import numpy as np
from processing import *

Get Jeopardy data and preprocess it

In [45]:
jeopardy_data = pd.read_csv("jeopardy.csv")
preprocess(jeopardy_data)

Ensure each category has 5 or more questions, and pick 7 random categories out of those available.

In [46]:
# Makes it easier to change the variables later
categories_to_show = 7
min_questions = 5

In [48]:
categories = jeopardy_data.category
categories = categories.value_counts().apply( \
    lambda c: c >= min_questions)
categories = categories[categories == True]
categories = categories.sample(categories_to_show)
categories = categories.index
categories

Index(['CNN 25: INTERNATIONAL NEWS', 'LET ME TAKE YOU DOWN',
       'AUTHORS BEFORE & AFTER', 'MGM WOULD HAVE CHANGED YOUR NAME',
       'NAME THAT REBELLION', 'MEDICAL PIONEERS', 'ON THE FLY'],
      dtype='object')

Get 5 questions for the available categories

In [49]:
questions = jeopardy_data[jeopardy_data.category.apply( \
    lambda c: c in categories)]
category_question_count = {}
def incr(key):
    category_question_count.setdefault(key, 0)
    category_question_count[key] += 1
    return category_question_count[key] <= 5
questions = questions[questions.category.apply(incr)].reset_index(drop=True)
questions

,show_number,air_date,round,category,value,question,answer,value_float,air_year
0,6128,2011-04-13,Jeopardy!,MGM WOULD HAVE CHANGED YOUR NAME,$200,Emil & Kjellfrid were the parents of this actr...,Renée Zellweger,200.0,2011
1,6128,2011-04-13,Jeopardy!,MGM WOULD HAVE CHANGED YOUR NAME,$400,Yvonne Strahovski plays a CIA agent on this TV...,Chuck,400.0,2011
2,6128,2011-04-13,Jeopardy!,MGM WOULD HAVE CHANGED YOUR NAME,$600,"This ""30 Rock"" actress changed her own name--i...",Jane Krakowski,600.0,2011
3,6128,2011-04-13,Jeopardy!,MGM WOULD HAVE CHANGED YOUR NAME,$800,"This singer/model ""Fifth Element"" star wants y...",Milla Jovovich,800.0,2011
4,6128,2011-04-13,Jeopardy!,MGM WOULD HAVE CHANGED YOUR NAME,$1000,This actress whose dad is from Senegal followe...,Gabourey Sidibe,1000.0,2011
5,4921,2006-01-23,Double Jeopardy!,MEDICAL PIONEERS,$400,"Life-changing & yummy too, the oral polio vacc...",Albert Sabin,400.0,2006
6,4921,2006-01-23,Double Jeopardy!,MEDICAL PIONEERS,$800,In 1901 this army surgeon who later had a D.C....,Walter Reed,800.0,2006
7,4921,2006-01-23,Double Jeopardy!,MEDICAL PIONEERS,$1200,This Italian anatomist's career went down the ...,Fallopius (or Gabriello Fallopio),1200.0,2006
8,4921,2006-01-23,Double Jeopardy!,MEDICAL PIONEERS,$1600,"His 1543 landmark work on anatomy, ""De Humani ...",Vesalius,1600.0,2006
9,4921,2006-01-23,Double Jeopardy!,MEDICAL PIONEERS,$2000,In 1949 the American Medical Women's Assoc. na...,Elizabeth Blackwell,2000.0,2006


Print out the categories and money values for each question in the category

In [50]:
# Terminals are typically 80 characters wide
view_width = 80
cell_width, expanded_cells = divmod(view_width, categories_to_show)
cell_width -= 1  # Add space for a pipe character
cell_template_str = "{{:^{}}}|"
cell_format_str = cell_template_str.format(cell_width)

In [51]:
longest_category = max(categories, key=len)
lines, extra_line = divmod(len(longest_category), cell_width)
if extra_line > 0:
    lines += 1

for line in range(lines):
    extender = expanded_cells
    for col in categories:
        start = line * cell_width
        end = start + cell_width
        print(cell_format_str.format(col[start:end]), sep="", end="")
        if extender > 0:
            print(" ", end="")
            extender -= 1
    print()
print("-" * view_width)  # Separator between categories and monies

CNN 25: IN| LET ME TAK| AUTHORS BE| MGM WOULD |NAME THAT |MEDICAL PI|ON THE FLY|
TERNATIONA| E YOU DOWN| FORE & AFT| HAVE CHANG|REBELLION |  ONEERS  |          |
  L NEWS  |           |     ER    | ED YOUR NA|          |          |          |
          |           |           |     ME    |          |          |          |
--------------------------------------------------------------------------------


Print out the money values for each selected question

In [68]:
for row in range(min_questions):
    extender = expanded_cells
    for col in categories:
        money_value = questions.loc[questions.category == col].value.iloc[row]
        money_number = questions.loc[questions.category == col].value_float.iloc[row]
        print(cell_format_str.format(money_value), end="", sep="")
        if extender > 0:
            print(" ", end="")
            extender -= 1
    print("\n", "-" * view_width, sep="")

   $200   |    $200   |    $400   |    $200   |   $400   |   $400   |   $100   |
--------------------------------------------------------------------------------
   $400   |    $400   |    $800   |    $400   |   $800   |   $800   |   $200   |
--------------------------------------------------------------------------------
   $600   |    $600   |   $1200   |    $600   |  $1200   |  $1200   |   $300   |
--------------------------------------------------------------------------------
   $800   |    $800   |   $1600   |    $800   |  $1600   |  $1600   |   $400   |
--------------------------------------------------------------------------------
  $1000   |   $1000   |   $2000   |   $1000   |  $2000   |  $2000   |   $500   |
--------------------------------------------------------------------------------
